In [1]:
import pandas as pd
import numpy as np
from copy import copy
import random
import re

# Генератор имен

Генератор набора данных для имен.

Данные были получены с сайта https://ru.wikinews.org

 - [Мужские имена](https://ru.wikinews.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A0%D1%83%D1%81%D1%81%D0%BA%D0%B8%D0%B5_%D0%BC%D1%83%D0%B6%D1%81%D0%BA%D0%B8%D0%B5_%D0%B8%D0%BC%D0%B5%D0%BD%D0%B0_%D0%BF%D0%BE_%D0%B0%D0%BB%D1%84%D0%B0%D0%B2%D0%B8%D1%82%D1%83)
 - [Мужские фамилии](https://ru.wikinews.org/w/index.php?title=%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A0%D1%83%D1%81%D1%81%D0%BA%D0%B8%D0%B5_%D0%BC%D1%83%D0%B6%D1%81%D0%BA%D0%B8%D0%B5_%D1%84%D0%B0%D0%BC%D0%B8%D0%BB%D0%B8%D0%B8_%D0%BF%D0%BE_%D0%B0%D0%BB%D1%84%D0%B0%D0%B2%D0%B8%D1%82%D1%83&subcatuntil=%D0%9A%D0%B0%D0%BB%D0%B8%D0%BD%D0%B8%D0%BD+%28%D1%84%D0%B0%D0%BC%D0%B8%D0%BB%D0%B8%D1%8F%29#mw-subcategories)
 - [Женские имена](https://ru.wikinews.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A0%D1%83%D1%81%D1%81%D0%BA%D0%B8%D0%B5_%D0%B6%D0%B5%D0%BD%D1%81%D0%BA%D0%B8%D0%B5_%D0%B8%D0%BC%D0%B5%D0%BD%D0%B0_%D0%BF%D0%BE_%D0%B0%D0%BB%D1%84%D0%B0%D0%B2%D0%B8%D1%82%D1%83)
 - [Женские фамилии](https://ru.wikinews.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A0%D1%83%D1%81%D1%81%D0%BA%D0%B8%D0%B5_%D0%B6%D0%B5%D0%BD%D1%81%D0%BA%D0%B8%D0%B5_%D1%84%D0%B0%D0%BC%D0%B8%D0%BB%D0%B8%D0%B8_%D0%BF%D0%BE_%D0%B0%D0%BB%D1%84%D0%B0%D0%B2%D0%B8%D1%82%D1%83)

Отчества были получены через модификацию мужских имен - добавления окончания "ович/евич" для мужских отчеств и "овна/евна" для жениских. Вручную были исправлены неверные модификации.

In [ ]:
m_names = pd.read_excel("data/names/train_data.xlsx", "masc_names")['М_Имя'].to{repl}list()
m_surnames = pd.read_excel("data/names/train_data.xlsx", "masc_surnames")['М_Фамилии'].to_list()
m_patnames = pd.read_excel("data/names/train_data.xlsx", "masc_patnames")['М_Отчества'].to_list()
m_names_arrays = {"names": m_names, 
                  "surnames": m_surnames, 
                  "patronymics": m_patnames}

f_names = pd.read_excel("data/names/train_data.xlsx", "fem_names")['Ж_Имя'].to_list()
f_surnames = pd.read_excel("data/names/train_data.xlsx", "fem_surnames")['Ж_Фамилии'].to_list()
f_patnames = pd.read_excel("data/names/train_data.xlsx", "fem_patnames")["Ж_Отчетства"].to_list()
f_names_arrays = {"names": f_names, 
                  "surnames": f_surnames, 
                  "patronymics": f_patnames}

Так как в корректируемых ячейках ничего, кроме ФИО, не встречается, то нулевой токен можно опустить.

In [ ]:
ner_tags = ['PER-NAME', 'PER_SURN', 'PER-PATR']

Создадим функцию для генерации имени из списков выше.

In [ ]:
def compose_name() -> tuple:

    """
    Функция для генерации имени.

    Возвращает кортеж из двух списков: слов и токенов имени.
    """

    name = dict()
    namesList = []

    # генерируем параметры для имени 
    sex = np.random.choice(["m", "f"], 1)[0]

    # print(sex)

    n_surnames = np.random.choice(6, size=1, p = [0.01, 0.5, 0.2, 0.2, 0.05, 0.04])[0]
    add_name = np.random.choice(2, size = 1, p = [0.05, 0.95])[0]
    add_patronymic = np.random.choice(2, size = 1, p = [0.05, 0.95])[0]

    if n_surnames == 0 and add_name == 0 and add_patronymic == 0:
        n_surnames = 1
        add_name = 1
        add_patronymic = 1

    # смешиваем порядок слов в имени, чтобы модель училась находить токены без привязки к его позиции.
    namesparts_order = np.array([0]*add_name + [1]*n_surnames + [2]*add_patronymic) + 1
    np.random.shuffle(namesparts_order)

    # print(namesparts_order)

    # генерирруем имя и токены
    if sex == "m":
        names_arrays = m_names_arrays
    else:
        names_arrays = f_names_arrays

    name[0] = np.random.choice(names_arrays["names"], size = add_name)

    name[1] = np.random.choice(names_arrays["surnames"], n_surnames, False)

    name[2] = np.random.choice(names_arrays["patronymics"], size = add_patronymic)

    for idx in namesparts_order:

        if idx == 1:
            namesList.append(name[0][0])
        
        if idx == 2:
            namesList.append(name[1][n_surnames - 1])
            n_surnames -= 1

        if idx == 3:
            namesList.append(name[2][0])

    return((namesList, namesparts_order))

In [ ]:
dataset = pd.DataFrame([compose_name() for i in range(40000)], columns=["tokens", "ner_tags"])

In [ ]:
dataset.to_json("data/names/names_train_large.json")
dataset.head()

# Генератор адресов

Данные для адресов получены с сайта [Федеральный информационной адресной системы](https://fias.nalog.ru/Frontend). 
Данные выгружались в формате [КЛАДР](https://kladr-rf.ru/). 

In [2]:
from simpledbf import Dbf5

SQLalchemy is not installed. No support for SQL output.


In [3]:
dbf = Dbf5('data/addresses/base/SOCRBASE.DBF', 'cp866')
SOCR_data = dbf.to_dataframe()

SOCR_data

,LEVEL,SCNAME,SOCRNAME,KOD_T_ST
0,1,Аобл,Автономная область,102
1,1,АО,Автономный округ,101
2,1,г,Город,103
3,1,г.ф.з.,Город федерального значения,112
4,1,край,Край,104
...,...,...,...,...
223,5,ф/х,Фермерское хозяйство,589
224,5,х,Хутор,558
225,5,ш,Шоссе,531
226,5,ю.,Юрты,931


In [4]:
SOCR_data[SOCR_data["LEVEL"] == "5"][:20]

,LEVEL,SCNAME,SOCRNAME,KOD_T_ST
119,5,аал,Аал,934
120,5,а/я,Абонентский ящик,572
121,5,аллея,Аллея,501
122,5,арбан,Арбан,936
123,5,аул,Аул,937
124,5,балка,Балка,581
125,5,берег,Берег,901
126,5,бугор,Бугор,576
127,5,б-р,Бульвар,502
128,5,вал,Вал,568


In [5]:
levels = {"level"+str(i) : SOCR_data[SOCR_data['LEVEL'] == str(i+1)]["SCNAME"].tolist() for i in range(SOCR_data['LEVEL'].nunique())}
[print(value) for key, value in levels.items()]

['Аобл', 'АО', 'г', 'г.ф.з.', 'край', 'обл', 'округ', 'Респ', 'Чувашия']
['АО', 'вн.тер. г.', 'г', 'г.', 'г.о.', 'м.р-н', 'п', 'р-н', 'тер', 'у']
['волость', 'г', 'дп', 'кп', 'массив', 'п', 'пгт', 'п/о', 'рп', 'рп.', 'с', 'с.', 'с/а', 'с/о', 'с/мо', 'с/п', 'с/с', 'тер']
['аал', 'автодорога', 'арбан', 'аул', 'волость', 'в-ки', 'высел', 'г', 'г-к', 'гп', 'гп.', 'дп', 'дп.', 'д', 'д.', 'ж/д_оп', 'ж/д_будка', 'ж/д в-ка', 'ж/д_казарм', 'ж/д_платф', 'ж/д пл-ка', 'ж/д ст', 'ж/д_ст', 'ж/д бл-ст', 'ж/д к-т', 'ж/д о.п.', 'ж/д_пост', 'ж/д п.п.', 'ж/д рзд', 'ж/д_рзд', 'жилзона', 'жилрайон', 'з-ка', 'заимка', 'зим.', 'казарма', 'кв-л', 'киш.', 'кордон', 'кп', 'кп.', 'лпх', 'массив', 'м', 'м-ко', 'мкр', 'нп', 'нп.', 'остров', 'пл.р-н', 'погост', 'п', 'п.', 'пгт', 'п/ст', 'п. ж/д ст.', 'п. ст.', 'пос.рзд', 'пос.рзд.', 'п-к', 'починок', 'п/о', 'промзона', 'рп', 'рп.', 'рзд', 'рзд.', 'снт', 'с', 'с.', 'сп', 'сп.', 'сл', 'сл.', 'ст-ца', 'ст', 'ст.', 'тер', 'у', 'у.', 'х', 'х.']
['аал', 'а/я', 'аллея', '

[None, None, None, None, None, None]

In [6]:
dbf = Dbf5('data/addresses/base/STREET.DBF', 'cp866')
STREET_data = dbf.to_dataframe()

# если на конце код 99 - адрес больше не актуален
STREET_data["STATUS"] = (STREET_data["CODE"].str[-2:] == "99").astype(int)

# улицы всегда 5го уровня
STREET_data["LEVEL"] = "5"

# объединим сокращения
streets = pd.merge(STREET_data, SOCR_data.iloc[:, :3], how = "left", left_on=['LEVEL', "SOCR"], right_on = ["LEVEL", "SCNAME"])

streets.head()

,NAME,SOCR,CODE,INDEX,GNINMB,UNO,OCATD,STATUS,LEVEL,SCNAME,SOCRNAME
0,Абадзехская,ул,01000001000000100,NaN,0100,NaN,79401000000,0,5,ул,Улица
1,Абрикосовая,ул,01000001000000200,NaN,0100,NaN,79401000000,0,5,ул,Улица
2,Авиационный,пер,01000001000000300,NaN,0100,NaN,79401000000,0,5,пер,Переулок
3,Автодорога 7,ул,01000001000000400,NaN,0100,NaN,79401000000,0,5,ул,Улица
4,Адыгейская,ул,01000001000000500,NaN,0100,NaN,79401000000,0,5,ул,Улица


In [7]:
dbf = Dbf5('data/addresses/base/DOMA.DBF', 'cp866')
HOUSE_data = dbf.to_dataframe()

# если на конце код 99 - адрес больше не актуален
HOUSE_data["STATUS"] = (HOUSE_data["CODE"].str[-2:] == "99").astype(int)

# дома всегда 6го уровня
HOUSE_data["LEVEL"] = "6"

# объединим сокращения
houses = pd.merge(HOUSE_data, SOCR_data.iloc[:, :3], how = "left", left_on=['LEVEL', "SOCR"], right_on = ["LEVEL", "SCNAME"])

houses.head()

,NAME,KORP,SOCR,CODE,INDEX,GNINMB,UNO,OCATD,STATUS,LEVEL,SCNAME,SOCRNAME
0,"11В,15Б,15Дстр1,15Лстр1,17,17А,17Д,23,43",NaN,ДОМ,0100000100000030001,385000,0100,NaN,79401000000,0,6,ДОМ,Дом
1,"4Б,8,8Астр10,8Астр426",NaN,ДОМ,0100000100000030002,385000,0100,NaN,79401000000,0,6,ДОМ,Дом
2,"1,10,103,12,15,16,20,22,3,39,39А,47А",NaN,ДОМ,0100000100000030003,385006,0100,NaN,79401000000,0,6,ДОМ,Дом
3,"4Б/1,4Бстр10,4Бстр11,4Бстр12,4Бстр13",NaN,ДОМ,0100000100000030004,385006,0100,NaN,79401000000,0,6,ДОМ,Дом
4,"4Бстр14,4Бстр15,4Бстр16,4Бстр17,4Бстр18",NaN,ДОМ,0100000100000030005,385006,0100,NaN,79401000000,0,6,ДОМ,Дом


In [8]:
dbf = Dbf5('data/addresses/base/KLADR.DBF', 'cp866')
adr_data = dbf.to_dataframe()

adr_data.head()

,NAME,SOCR,CODE,INDEX,GNINMB,UNO,OCATD,STATUS
0,Адыгея,Респ,0100000000000,NaN,0100,NaN,79000000000,0
1,Майкоп,г,0100000100000,NaN,0100,NaN,79401000000,0
2,Веселый,х,0100000100200,NaN,0100,NaN,79401000006,0
3,Гавердовский,х,0100000100300,NaN,0100,NaN,79401000002,0
4,Подгорный,п,0100000100400,NaN,0100,NaN,79401000004,0


Для генерации адреса мы провели предобработку данных: выделили коды для каждого уровня субъекта. 
Таким образом мы выделили:

 - Уровень 1: регион
 - Уровень 2: район региона
 - Уровень 3: населенный пункт (город, село, деревня)
 - Уровень 4: район населенного пункта
 - Уровень 5: улица
 - Уровень 6: дом
 - Уровень 7: квартира

In [9]:
# выделим коды
adr_data["LVL1_CODE"] = adr_data["CODE"].str[:2]
adr_data["LVL2_CODE"] = adr_data["CODE"].str[2:5]
adr_data["LVL3_CODE"] = adr_data["CODE"].str[5:8]
adr_data["LVL4_CODE"] = adr_data["CODE"].str[8:11]

# определим уровень каждого объекта
adr_data["LEVEL"] = np.where(adr_data["CODE"].str[2:] == "00000000000", 
                             "1", 
                             np.where((adr_data["CODE"].str[5:] == "00000000") & (adr_data["LVL2_CODE"] != "000"), 
                                      "2",
                                      np.where((adr_data["CODE"].str[8:] == "00000") & (adr_data["LVL3_CODE"] != "000"),
                                               "3",
                                               np.where((adr_data["CODE"].str[11:] == "00") & (adr_data["LVL4_CODE"] != "000"),
                                                        "4",
                                                        "99"))))

# если на конце код 99 или уровень 99 - адрес больше не актуален
adr_data["STATUS"] = ((adr_data["LEVEL"] == "99") | (adr_data["CODE"].str[11:] == "99")).astype(int)

# объединим сокращения
addresses = pd.merge(adr_data, SOCR_data.iloc[:, :3], how = "left", left_on=['LEVEL', "SOCR"], right_on = ["LEVEL", "SCNAME"])

In [10]:
addresses.head()

,NAME,SOCR,CODE,INDEX,GNINMB,UNO,OCATD,STATUS,LVL1_CODE,LVL2_CODE,LVL3_CODE,LVL4_CODE,LEVEL,SCNAME,SOCRNAME
0,Адыгея,Респ,0100000000000,NaN,0100,NaN,79000000000,0,01,000,000,000,1,Респ,Республика
1,Майкоп,г,0100000100000,NaN,0100,NaN,79401000000,0,01,000,001,000,3,г,Город
2,Веселый,х,0100000100200,NaN,0100,NaN,79401000006,0,01,000,001,002,4,х,Хутор
3,Гавердовский,х,0100000100300,NaN,0100,NaN,79401000002,0,01,000,001,003,4,х,Хутор
4,Подгорный,п,0100000100400,NaN,0100,NaN,79401000004,0,01,000,001,004,4,п,Поселок


In [11]:
addresses[(addresses['LVL1_CODE'] == "27") & (addresses['LEVEL'] == "2")]

,NAME,SOCR,CODE,INDEX,GNINMB,UNO,OCATD,STATUS,LVL1_CODE,LVL2_CODE,LVL3_CODE,LVL4_CODE,LEVEL,SCNAME,SOCRNAME
45271,Амурский,р-н,2700200000000,NaN,2700,NaN,08203000000,0,27,002,000,000,2,р-н,Район
45323,Аяно-Майский,р-н,2700300000000,NaN,2700,2720,08206000000,0,27,003,000,000,2,р-н,Район
45346,Бикинский,р-н,2700400000000,NaN,2700,2707,08209000000,0,27,004,000,000,2,р-н,Район
45380,Ванинский,р-н,2700500000000,NaN,NaN,NaN,08212000000,0,27,005,000,000,2,р-н,Район
45424,Верхнебуреинский,р-н,2700600000000,NaN,NaN,NaN,08214000000,0,27,006,000,000,2,р-н,Район
45500,Вяземский,р-н,2700700000000,NaN,2700,2720,08217501000,0,27,007,000,000,2,р-н,Район
45545,Имени Лазо,р-н,2700800000000,NaN,2700,2720,08224000000,0,27,008,000,000,2,р-н,Район
45604,Имени Полины Осипенко,р-н,2700900000000,NaN,2700,2728,08237000000,0,27,009,000,000,2,р-н,Район
45621,Комсомольский,р-н,2701000000000,NaN,2700,2728,08220000000,0,27,010,000,000,2,р-н,Район
45803,Нанайский,р-н,2701100000000,NaN,2700,2720,08228000000,0,27,011,000,000,2,р-н,Район


In [12]:
addresses[(addresses['LVL2_CODE'] != "000") & (addresses['LVL3_CODE'] != "000") & (addresses['STATUS'] == 0)]

,NAME,SOCR,CODE,INDEX,GNINMB,UNO,OCATD,STATUS,LVL1_CODE,LVL2_CODE,LVL3_CODE,LVL4_CODE,LEVEL,SCNAME,SOCRNAME
841,Искинский,с/с,0200100300000,NaN,0274,NaN,80401375000,0,02,001,003,000,3,с/с,Сельсовет
842,Нагаевский,с/с,0200100400000,NaN,0276,NaN,80401944000,0,02,001,004,000,3,с/с,Сельсовет
847,Булгаковский,с/с,0200100900000,NaN,0272,0245,80252810000,0,02,001,009,000,3,с/с,Сельсовет
848,Дмитриевский,с/с,0200101000000,NaN,0272,0245,80252815000,0,02,001,010,000,3,с/с,Сельсовет
850,Зубовский,с/с,0200101200000,NaN,0272,0245,80252820000,0,02,001,012,000,3,с/с,Сельсовет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252761,Нововоронцовка,пгт,9501800100000,NaN,9500,NaN,74237551000,0,95,018,001,000,3,пгт,Поселок городского типа
252806,Новотроицкое,пгт,9501900100000,NaN,9500,NaN,74239551000,0,95,019,001,000,3,пгт,Поселок городского типа
252838,Алешки,г,9502000100000,NaN,9500,NaN,74203501000,0,95,020,001,000,3,г,Город
252879,Верхний Рогачик,пгт,9502100100000,NaN,9500,NaN,74213551000,0,95,021,001,000,3,пгт,Поселок городского типа


Так как в некоторых ячейках формы, помимо адресов, могут содержаться и другие слова ("Проживал в...", "переехал из" и т.д.), то при обучении модели необходимо также предусмотреть и нулевой токен.

In [13]:
label_names = ["O",
            "LOC-REG", 
            "LOC-DIST", 
            "LOC-SETL", 
            "LOC-CDIST", 
            "LOC-STRT", 
            "LOC-HOUS", 
            "LOC-FLAT"]

In [14]:
def generate_address():

    """
    Функция для генерации "настоящего" адреса, то есть города и улицы выбираются из соответствующих регионов и городов.

    Возвращает:
        Кортеж из 3 элементов: строку с адресом, порядок токенов и порядок кодов
    """

    # выбор региона (уровень 1)

    lvls = []
    address_parts = []
    tbl = copy(addresses)
    addressparts_order = []

    for i in range(4):

        lvlcode = "LVL"+str(i+1)+"_CODE"

        probs = (tbl[lvlcode].value_counts()/np.sum(tbl[lvlcode].value_counts())).tolist()
        lvls.append(np.random.choice(tbl[lvlcode].unique().tolist(), 1, probs)[0])

        tbl = tbl[tbl[lvlcode] == lvls[-1]]

        fullmode = np.random.choice([True, False], 1)
        line = tbl[(tbl[lvlcode] == lvls[-1]) & (tbl['LEVEL'] == str(i+1))]

        if not line.empty:

            if fullmode:
                address_parts.append(line['NAME'].values[0] + " " + line['SOCRNAME'].values[0])
            else:
                address_parts.append(line['NAME'].values[0] + " " + line['SOCR'].values[0])
            
            addressparts_order.extend([i+1, i+1])

    # выбрать улицу (если имеется)
    streets_set = streets[streets["CODE"].str.startswith("".join(lvls)+"00")]

    if streets_set["CODE"].tolist() != []:

        probs = (streets_set["CODE"].value_counts()/np.sum(streets_set["CODE"].value_counts())).tolist()
        lvls.append(np.random.choice(streets_set["CODE"].tolist(), 1, probs)[0])

        fullmode = np.random.choice([True, False], 1)
        socr_position_front = np.random.choice([True, False], 1)

        line = streets_set[streets_set['CODE'] == lvls[-1]]

        if fullmode:
            if socr_position_front:
                address_parts.append(line['SOCRNAME'].values[0] + " " + line['NAME'].values[0])
                addressparts_order.extend([10, 9])
            else:
                address_parts.append(line['NAME'].values[0] + " " + line['SOCRNAME'].values[0])
                addressparts_order.extend([9, 10])
        else:
            if socr_position_front:
                address_parts.append(line['SOCR'].values[0] + " " + line['NAME'].values[0])
                addressparts_order.extend([10, 9])
            else:
                address_parts.append(line['NAME'].values[0] + " " + line['SOCR'].values[0])
                addressparts_order.extend([9, 10])


        # Выбрать дом
        houses_set = houses[houses["CODE"].str.startswith(lvls[-1])]

        if houses_set["CODE"].tolist() != []:
            
            probs = (houses_set["CODE"].value_counts()/np.sum(houses_set["CODE"].value_counts())).tolist()
            lvls.append(np.random.choice(houses_set["CODE"].tolist(), 1, probs)[0])

            fullmode = np.random.choice([True, False], 1)

            line = houses_set[houses_set['CODE'] == lvls[-1]]
            house = np.random.choice(line['NAME'].values[0].split(","), 1)[0]

            if fullmode:
                address_parts.append(line['SOCRNAME'].values[0] + " " + str(house))
            else:
                address_parts.append(line['SOCR'].values[0] + " " + str(house))

            addressparts_order.extend([11, 12])

    address = ", ".join(address_parts).replace("/", "")
        
    return address, addressparts_order, lvls

In [15]:
generate_address()

('Калмыкия Республика, Ики-Бурульский Район, Цаган Ташу п, ул Дружбы, ДОМ 1',
 [1, 1, 2, 2, 4, 4, 10, 9, 11, 12],
 ['08', '003', '000', '034', '08003000034000100', '0800300003400010001'])

Так как генерация правдоподобного адреса достаточно долгая, принято решение использовать другую функцию, которая делает меньше отборов (фильтраций), но может сочетать разные города и регионы, улицы, дома и тд.

In [14]:
def quick_generate_address():

    """
    Функция для быстрой генерации адреса. Элементы разного уровня могут не соответствовать друг-другу.

    Возвращает:
        Кортеж из 3 элементов: слова из строки с адресом (1 слово + сокращение - 1 элемент), порядок токенов и порядок кодов
    """

    # выбор региона (уровень 1)

    lvls = []
    address_parts = []
    tbl = copy(addresses)
    addressparts_order = []

    for i in range(4):

        lvlcode = "LVL"+str(i+1)+"_CODE"

        probs = (tbl[lvlcode].value_counts()/np.sum(tbl[lvlcode].value_counts())).tolist()
        lvls.append(np.random.choice(tbl[lvlcode].unique(), 1, probs)[0])

        fullmode = np.random.choice([True, False], 1)
        line = tbl[(tbl[lvlcode] == lvls[-1]) & (tbl['LEVEL'] == str(i+1))]

        if not line.empty:

            if fullmode:
                address_parts.append([line['NAME'].values[0], line['SOCRNAME'].values[0]])
            else:
                address_parts.append([line['NAME'].values[0], line['SOCR'].values[0]])
            
            addressparts_order.append([i+1, i+1])

    # выбрать улицу
    do_streethouse = np.random.choice([True, False], 1)

    if do_streethouse:
        lvls.append(np.random.choice(streets["CODE"], 1, probs)[0])

        fullmode = np.random.choice([True, False], 1)
        socr_position_front = np.random.choice([True, False], 1)

        line = streets[streets['CODE'] == lvls[-1]]

        if fullmode:
            if socr_position_front:
                address_parts.append([line['SOCRNAME'].values[0], line['NAME'].values[0]])
            else:
                address_parts.append([line['NAME'].values[0], line['SOCRNAME'].values[0]])
        else:
            if socr_position_front:
                address_parts.append([line['SOCR'].values[0], line['NAME'].values[0]])
            else:
                address_parts.append([line['NAME'].values[0], line['SOCR'].values[0]])

        addressparts_order.append([5, 5])

        # Выбрать дом
        lvls.append(np.random.choice(houses["CODE"], 1, probs)[0])

        fullmode = np.random.choice([True, False], 1)

        line = houses[houses['CODE'] == lvls[-1]]
        house = np.random.choice(line['NAME'].values[0].split(","), 1)[0]

        if fullmode:
            address_parts.append(["дом", str(house)])
        else:
            address_parts.append(["д.", str(house)])

        addressparts_order.append([6, 6])

        # Добавить квартиру
        lvls.append(lvls[-1])

        fullmode = np.random.choice([True, False], 1)

        flat = np.random.choice(list(range(1, 1024)), 1)[0]

        if fullmode:
            address_parts.append(["Квартира", str(flat)])
        else:
            address_parts.append(["кв.", str(flat)])

        addressparts_order.append([7, 7])


    # print(address_parts)
    # print(addressparts_order)
    # print(lvls)

    zipped = list(zip(address_parts, addressparts_order, lvls))
    random.shuffle(zipped)
    address_parts, addressparts_order, lvls = zip(*zipped)

    # address = ", ".join(address_parts).replace("/", "")

    address_parts = [elem for elems in address_parts for elem in elems]
    addressparts_order = [elem for elems in addressparts_order for elem in elems]
        
    return address_parts, addressparts_order, lvls

In [15]:
quick_generate_address()

(['Саратовская',
  'обл',
  'дом',
  '29',
  'кв.',
  '255',
  'Школьная',
  'Улица',
  'Куйвози',
  'Массив',
  'Штурбино',
  'Село',
  'Кармаскалинский',
  'р-н'],
 [1, 1, 6, 6, 7, 7, 5, 5, 3, 3, 4, 4, 2, 2],
 ('64',
  '6600000100005250002',
  '6600000100005250002',
  '68002000104001000',
  '073',
  '025',
  '029'))

Сгенерируем 1200 наблюдений.

In [ ]:
dataset_adr = pd.DataFrame([quick_generate_address() for i in range(1200)], columns=["tokens", "ner_tags", "levels"])
dataset_adr.to_json("data/addresses/addresses_train_med.json")
dataset_adr.head()

Также рассмотрим генерацию фраз, содержащих адреса, а также другую информацию (нулевые токены).

In [16]:
wordings = ["Дом находится по адресу {split}.", 
            "В 1964 году я пошёл в первый класс средней школы № 2, которая находилась по адресу {split}.",
            "Выставочный зал находится по адресу {split}.", 
            "{split} прибыл из {split}.",
            "По адресу {split} находится мастерская по производству кабардинских тканей.",
            "Семиэтажное здание находилось по адресу: {split}, ухоженный фасад дома выглядел хорошо.",
            "В 1996 году переехал из {split} в {split}.",
            "Проживала по адресу: {split}.",
            "{split} прибыла из {split}.",
            "Проживал по адресу: {split}.",
            "Как санитарные машины перестали наконец приезжать совсем, \
и на подъездах появились объявления «ближайший пункт экстренной помощи находится по адресу {split}, \
и люди сами, на санках везли туда своих заболевших, а потом и мёртвых. ",
            "Проживает по адресу: {split}.",
            "Проживает по адресу: {split}, ранее проживал в {split}.",
            "Здание, которое раньше находилось по адресу: {split}, было разрушено полностью.",
            "Гидрометео бюро находилось по адресу: {split}, но переехало в {split}.",
            "Мой дом находится по адресу {split}, а это не мой новый дом!"]

In [17]:
def quick_generate_address_text():

    """
    Функция для быстрой генерации фраз, содержащих адреса.

    Возвращает:
        Кортеж из 2 элементов: токенов (слов) и меток токенов
    """

    tokens = []
    ner_tags = []

    # случайным образом выбираем текстовку
    wording = np.random.choice(wordings, 1)[0]
    # print(wording)

    parts_breakdown = wording.strip().replace(".", "").split("{split}")

    # print(parts_breakdown, len(parts_breakdown))

    # на место {split} подставляем данные по адресам.
    for i, part in enumerate(parts_breakdown):

        # print(i, part)

        if part == "" and i == len(parts_breakdown)-1:

            continue

        elif part == "" and i != len(parts_breakdown)-1:

            # print("ADRESS! 1")
            address, tags, _ = quick_generate_address()
            tokens.extend(address)
            ner_tags.extend(tags)
        
        else:
            
            for word in part.split():

                tokens.append(word)
                ner_tags.append(0)

            if i < len(parts_breakdown)-1:
                
                # print("ADRESS! 2")
                address, tags, _ = quick_generate_address()
                tokens.extend(address)
                ner_tags.extend(tags)
        
    return tokens, ner_tags

In [18]:
quick_generate_address_text()

(['В',
  '1964',
  'году',
  'я',
  'пошёл',
  'в',
  'первый',
  'класс',
  'средней',
  'школы',
  '№',
  '2,',
  'которая',
  'находилась',
  'по',
  'адресу',
  'Татарстан',
  'Респ',
  'Нагорское',
  'Деревня',
  'Шумерля',
  'г'],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 4, 4, 3, 3])

In [73]:
dataset_adr = pd.DataFrame([quick_generate_address_text() for i in range(1200)], columns=["tokens", "ner_tags"])
dataset_adr.to_json("data/addresses/addresses_train_med_extra.json")
dataset_adr.head()

,tokens,ner_tags
0,"[Семиэтажное, здание, находилось, по, адресу:,...","[0, 0, 0, 0, 0, 4, 4, 1, 1, 2, 2, 3, 3, 0, 0, ..."
1,"[Проживает, по, адресу:, Нагорское, Деревня, д...","[0, 0, 0, 4, 4, 6, 6, 7, 7, 5, 5, 2, 2, 1, 1]"
2,"[Проживает, по, адресу:, Боханский, р-н, Тойси...","[0, 0, 0, 2, 2, 3, 3, 6, 6, 5, 5, 7, 7, 1, 1, ..."
3,"[Выставочный, зал, находится, по, адресу, Чишм...","[0, 0, 0, 0, 0, 2, 2, 1, 1, 3, 3, 4, 4]"
4,"[Проживает, по, адресу:, Солтонский, р-н, Стер...","[0, 0, 0, 2, 2, 3, 3, 1, 1, 4, 4, 0, 0, 0, 0, ..."
